In [1]:
 # Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pymongo
import pandas as pd
from splinter import Browser
import time

In [2]:
 # Initialize PyMongo to work with MongoDBs
# conn = 'mongodb://localhost:27017'
# client = pymongo.MongoClient(conn)

In [3]:
# URL of page to be scraped
url = 'https://dogtime.com/dog-breeds'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = bs(response.text, 'lxml')

In [4]:
# Retrieve the parent divs for all articles
results = soup.find_all('div', class_='list-item')
results

[<div class="list-item"><a class="list-item-title" href="https://dogtime.com/dog-breeds/afador">Afador</a></div>,
 <div class="list-item"><a class="list-item-title" href="https://dogtime.com/dog-breeds/affenhuahua">Affenhuahua</a></div>,
 <div class="list-item"><a class="list-item-title" href="https://dogtime.com/dog-breeds/affenpinscher">Affenpinscher</a></div>,
 <div class="list-item"><a class="list-item-title" href="https://dogtime.com/dog-breeds/afghan-hound">Afghan Hound</a></div>,
 <div class="list-item"><a class="list-item-title" href="https://dogtime.com/dog-breeds/airedale-terrier">Airedale Terrier</a></div>,
 <div class="list-item"><a class="list-item-title" href="https://dogtime.com/dog-breeds/akbash">Akbash</a></div>,
 <div class="list-item"><a class="list-item-title" href="https://dogtime.com/dog-breeds/akita">Akita</a></div>,
 <div class="list-item"><a class="list-item-title" href="https://dogtime.com/dog-breeds/akita-chow">Akita Chow</a></div>,
 <div class="list-item"><a

In [5]:
# loop over results to get article data

dog_breed_list = []
dog_links = []
for result in results:
    # scrape the dog types 
    dog_type = result.find('a', class_='list-item-title').text
    dog_breed_list.append(dog_type)
    
    #scrape the link
    link = result.a['href']
    dog_links.append(link)

In [6]:
#zip dog list and dog link lists together
combined_lists = list(zip(dog_breed_list, dog_links))


In [7]:
dog_info_df = pd.DataFrame(combined_lists)

In [8]:
dog_info_df = dog_info_df.rename(columns = {0:"Dog", 1:"Link"})
dog_info_df

,Dog,Link
0,Afador,https://dogtime.com/dog-breeds/afador
1,Affenhuahua,https://dogtime.com/dog-breeds/affenhuahua
2,Affenpinscher,https://dogtime.com/dog-breeds/affenpinscher
3,Afghan Hound,https://dogtime.com/dog-breeds/afghan-hound
4,Airedale Terrier,https://dogtime.com/dog-breeds/airedale-terrier
...,...,...
372,Whoodle,https://dogtime.com/dog-breeds/whoodle
373,Wirehaired Pointing Griffon,https://dogtime.com/dog-breeds/wirehaired-poin...
374,Xoloitzcuintli,https://dogtime.com/dog-breeds/xoloitzuintli
375,Yorkipoo,https://dogtime.com/dog-breeds/yorkipoo


In [9]:
test = dog_links[0:4]
test

['https://dogtime.com/dog-breeds/afador',
 'https://dogtime.com/dog-breeds/affenhuahua',
 'https://dogtime.com/dog-breeds/affenpinscher',
 'https://dogtime.com/dog-breeds/afghan-hound']

In [10]:
executable_path = {'executable_path': 'chromedriver.exe'}


In [11]:
#intialize dataframe row lists
#source: https://stackoverflow.com/questions/10715965/add-one-row-to-pandas-dataframe
rows_list = []
#open browser
browser = Browser('chrome', **executable_path, headless=False)

#visit each link and extract rating data
for dog_link in test:
    #initialize list
    stars_list = []
    # Visit each dogtime url
    browser.visit(dog_link)
    
     # Scrape page into Soup
    html2 = browser.html
    soup2 = bs(html2, "lxml")
    
    dog_name_finder = soup2.find_all('div', class_= "breeds-single-content")[0]
    #find breed and append to list
    breed = dog_name_finder.find('h1').text
    stars_list.append(breed)
    
    #find area of page with details about star ratings
    check = soup2.find_all('div', class_= "characteristic-stars parent-characteristic")

    #iterate through each category and get rating
    for dog_type in check:
        #find star rating
        star = str(dog_type.find('div'))
        #add star rating to list
        stars_list.append(star)
    #add breed/star list to dataframe rows list
    rows_list.append(stars_list)

    

In [13]:
#create dataframe with all row lists
dog_trait_df = pd.DataFrame(rows_list)   

In [14]:
dog_trait_df

,0,1,2,3,4,5
0,Afador,"<div class=""characteristic-star-block""><div cl...","<div class=""characteristic-star-block""><div cl...","<div class=""characteristic-star-block""><div cl...","<div class=""characteristic-star-block""><div cl...","<div class=""characteristic-star-block""><div cl..."
1,Affenhuahua,"<div class=""characteristic-star-block""><div cl...","<div class=""characteristic-star-block""><div cl...","<div class=""characteristic-star-block""><div cl...","<div class=""characteristic-star-block""><div cl...","<div class=""characteristic-star-block""><div cl..."
2,Affenpinscher,"<div class=""characteristic-star-block""><div cl...","<div class=""characteristic-star-block""><div cl...","<div class=""characteristic-star-block""><div cl...","<div class=""characteristic-star-block""><div cl...","<div class=""characteristic-star-block""><div cl..."
3,Afghan Hound,"<div class=""characteristic-star-block""><div cl...","<div class=""characteristic-star-block""><div cl...","<div class=""characteristic-star-block""><div cl...","<div class=""characteristic-star-block""><div cl...","<div class=""characteristic-star-block""><div cl..."


In [15]:
#clean up ratings columns
#souce: https://datatofish.com/replace-values-pandas-dataframe/
#source: https://thispointer.com/pandas-loop-or-iterate-over-all-or-certain-columns-of-a-dataframe/#:~:text=iteritems()%20i.e.-,DataFrame.iteritems(),and%20column%20contents%20as%20series.&text=As%20there%20were%203%20columns%20so%203%20tuples%20were%20returned%20during%20iteration.
for column in dog_trait_df:
    dog_trait_df[column] = dog_trait_df[column].replace(['<div class="characteristic-star-block"><div class="star star-1"></div></div>'],1)
    dog_trait_df[column] = dog_trait_df[column].replace(['<div class="characteristic-star-block"><div class="star star-2"></div></div>'],2)
    dog_trait_df[column] = dog_trait_df[column].replace(['<div class="characteristic-star-block"><div class="star star-3"></div></div>'],3)
    dog_trait_df[column] = dog_trait_df[column].replace(['<div class="characteristic-star-block"><div class="star star-4"></div></div>'],4)
    dog_trait_df[column] = dog_trait_df[column].replace(['<div class="characteristic-star-block"><div class="star star-5"></div></div>'],5)

In [16]:
#rename columns
dog_trait_df = dog_trait_df.rename(columns = {0:"Breed",
                                             1:"Adaptability",
                                             2:"All Around Friendliness",
                                             3:"Health and Grooming Needs",
                                             4:"Trainability",
                                             5:"Physical Needs"})

In [17]:
#set breed as index
dog_trait_df.set_index('Breed', inplace = True)

In [18]:
dog_trait_df

,Adaptability,All Around Friendliness,Health and Grooming Needs,Trainability,Physical Needs
Breed,,,,,
Afador,2,3,3,4,4
Affenhuahua,3,3,3,3,3
Affenpinscher,3,3,2,3,4
Afghan Hound,4,4,2,3,4
